In [1]:
import pandas as pd
import os

In [2]:
"""Functions Related to Tranfering Data to Excel Sheet"""
def writeDataframe6(wbpath,df,sheetname,dfcols,xlcols,indexcol):    
    wb=opxl.load_workbook(wbpath)
    wsheet=wb[sheetname]    
    for index,row in df.iterrows():        
        myrow=int(row[indexcol])
        for idf,ixl in zip(dfcols,xlcols):
            wsheet.cell(row=myrow,column=ixl).value=row[idf]               
    wb.save(wbpath)
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()  

In [3]:
"""Packagewise and Structurewise Summation"""
def packageWiseSum(package_df,projection_df):
    plist=list(package_df['Package_No'])
    for index,row in projection_df.iterrows():
        pname=row['Package']
        row_index=plist.index(pname)
        package_df.iloc[row_index,1]=package_df.iloc[row_index,1]+row['VDW_30_06_2020']
        package_df.iloc[row_index,2]=package_df.iloc[row_index,2]+row['VDW_30_06_2021']
    for index,row in package_df.iterrows():
        package_df.iloc[index,4]= package_df.iloc[index,2]- package_df.iloc[index,3]
    return package_df
def structureWiseSum(structure_df,projection_df):
    plist=list(structure_df['Code'])
    for index,row in projection_df.iterrows():
        pname=row['Structure Code']
        row_index=plist.index(pname)
        structure_df.iloc[row_index,3]=structure_df.iloc[row_index,3]+row['VDW_30_06_2020']
        structure_df.iloc[row_index,4]=structure_df.iloc[row_index,4]+row['VDW_30_06_2021']
    for index,row in structure_df.iterrows():
        structure_df.iloc[index,6]= structure_df.iloc[index,4]- structure_df.iloc[index,5]
    return structure_df

def dppWiseSum(structure_df,dpp_df):    
    dpp_item=list(dpp_df['DPP_Item']) #DPP_Item
    for index,row in structure_df.iterrows():
        #item_name=row[]
        pname=row['DPP_Item']
        row_index= dpp_item.index(pname)
        dpp_df.iloc[row_index,1]= dpp_df.iloc[row_index,1]+row['VWD_30_06_2020']
        dpp_df.iloc[row_index,2]=dpp_df.iloc[row_index,2]+row['VWD_30_06_2021']
    for index,row in dpp_df.iterrows():
        dpp_df.iloc[index,4]= dpp_df.iloc[index,2]- dpp_df.iloc[index,3]
       
    return dpp_df



In [4]:
def buildPackageWisePaymentMatrix(package_df):
    #Package_No Payment_30_06_2020
    package=list(package_df['Package_No'])
    paid=list(package_df['Payment_30_06_2020'])
    zero_list=[0 for x in package]
    data={'Package_No':package,'VWD_30_06_2020':zero_list,'VWD_30_06_2021':zero_list,
          'Payment_30_06_2020':paid,'Payment_30_06_2021':zero_list}
    package_payment_df=pd.DataFrame(data)
    return package_payment_df
def buildStructureWisePaymentMatrix(structure_df):
    #DPP Item	Initial	Code
    structure_name=list(structure_df['Items'])
    structure_code=list(structure_df['Code'])
    DPP_Item_code=list(structure_df['DPP_Item'])  
    payment=list(structure_df['Payment_30_06_2020'])
    #Package_No Payment_30_06_2020    
    zero_list=[0 for x in structure_name]
    data={'Items':structure_name,'Code':structure_code ,'DPP_Item':DPP_Item_code,'VWD_30_06_2020':zero_list,'VWD_30_06_2021':zero_list,
          'Payment_30_06_2020': payment,'Payment_30_06_2021':zero_list}
    structure_payment_df=pd.DataFrame(data)
    return structure_payment_df
    
def buildVWDMatrix(projection_df):
    #Component	Unit	Qnty	Cost	Present_Progress	Projected_progress
    #Package
    #Structure Code
    stc=list(projection_df['Structure Code'])
    packages=list(projection_df['Package'])
    components=list(projection_df['Component'])
    units=list(projection_df['Unit'])
    quantity=list(projection_df['Qnty'])
    cost=list(projection_df['Cost'])
    present_prog=list(projection_df['Present_Progress'])
    projected_prog=list(projection_df['Projected_progress'])
    zero_list=[0 for x in components ]
    data={'Package':packages,'Structure Code':stc,'Component':components,'Unit':units,'Qnty':quantity,'Cost':cost,
          'Present_Progress':present_prog,'Projected_progress':projected_prog,
          'VDW_30_06_2020': zero_list,'VDW_30_06_2021':zero_list,'Work_Done_20_21':zero_list,'Payment_30_06_2020': zero_list,
          'Payment_30_06_2020':zero_list}
    vwd_mat=pd.DataFrame(data)
    for index,row in vwd_mat.iterrows():
        print("index={}".format(index))
        vwd_mat.iloc[index,8]=vwd_mat.iloc[index,5]*vwd_mat.iloc[index,6]
        vwd_mat.iloc[index,9]=vwd_mat.iloc[index,5]*vwd_mat.iloc[index,7]
        vwd_mat.iloc[index,10]=vwd_mat.iloc[index,9]-vwd_mat.iloc[index,8]
    return vwd_mat
def buildPhysicalProgressMatrix(projection_df,str_code_df,package_names_df,progress_col_name,result):
    str_code_list=list(str_code_df['Code'])
    str_name_list=list(str_code_df['Items'])
    pname=list(package_names_df['Package_No'])
    offset=1
    for index,row in projection_df.iterrows():
        #Package Name	Component	Unit	Qnty	Cost	Present_Progress	Projected_progress	Package	Haor Type	Structure Code	Earth Volume	Hoar Code	District Code

        stc_code=row['Structure Code']
        pack_name=row['Package']
        qnty=row['Qnty']
        pval=row[progress_col_name]
        qval=qnty*pval
        row_index=pname.index(pack_name)
        col_index=str_code_list.index(stc_code)+offset
        result.iloc[row_index,col_index]=result.iloc[row_index,col_index]+qval

    return result
def buildPhysicalWorkPrgramCurrentYear(present_year,last_year):
    wp=present_year.copy(deep=True)
    #mycolumns=list(wp.columns)
    
    for index,row in wp.iterrows():
        for i in range(1,16):
            #wp.iloc[index,i]=0
            wp.iloc[index,i]=present_year.iloc[index,i]-last_year.iloc[index,i]
    return wp
            
        
    
   
    


In [5]:
myfolder=r'E:\Website_24_11_2020\cmis6\cmis6\Civilworks cost\RADP Preparations' #Home Computer
#myfolder=r'F:\website\cmis6\Civilworks cost\RADP_20_21'
input_file_name=os.path.join(myfolder,'Projecttion_input.xlsx')
out_file_name=os.path.join(myfolder,'Projecttion_output.xlsx')
myframes=[]
mynames=[]
sheetName="Structure_Type"
structure_type_df=pd.read_excel(input_file_name,sheet_name=sheetName)
structure_type_df.fillna(0,inplace=True)
structure_type_df
myframes.append(structure_type_df)
mynames.append("Structure_Type")
"""Projection Info"""
sheetName="Kishoregnj"
#sheetName="Netrokona"
#sheetName="Hobiganj"
#sheetName="Sunamgonj"
projection_info_df=pd.read_excel(input_file_name,sheet_name=sheetName)
projection_info_df.fillna(0,inplace=True)
projection_info_df
myframes.append(projection_info_df)
mynames.append("Projection_Info")
sheetName="Packages"
package_name_df=pd.read_excel(input_file_name,sheet_name=sheetName)
package_name_df.fillna(0,inplace=True)
myframes.append(package_name_df)
mynames.append("Package_Name")
vwd_df=buildVWDMatrix(projection_info_df)
myframes.append(vwd_df)
mynames.append("Value_of_Work_Done")
package_payment_df=buildPackageWisePaymentMatrix(package_name_df)
package_payment_df=packageWiseSum(package_payment_df,vwd_df)
myframes.append(package_payment_df)
mynames.append("Package_wise_payment")
"""Building Structurewise Cost"""
structure_payment_df=buildStructureWisePaymentMatrix(structure_type_df)
structure_payment_df=structureWiseSum(structure_payment_df,vwd_df)
myframes.append(structure_payment_df)
mynames.append("Structure_Wise_payment")
"""Building Structurewise Cost"""
sheetName="DPP_Item"
dpp_item_df=pd.read_excel(input_file_name,sheet_name=sheetName)
dpp_item_df.fillna(0,inplace=True)
dpp_item_df=dppWiseSum(structure_payment_df,dpp_item_df)
myframes.append(dpp_item_df)
mynames.append("DPP_Wise_payment")
#saveDataFrame(myframes,out_file_name,mynames)



index=0
index=1
index=2
index=3
index=4
index=5
index=6
index=7
index=8
index=9
index=10
index=11
index=12
index=13
index=14
index=15
index=16
index=17
index=18
index=19
index=20
index=21
index=22
index=23
index=24
index=25
index=26
index=27
index=28
index=29
index=30
index=31
index=32
index=33
index=34
index=35
index=36
index=37
index=38
index=39
index=40
index=41
index=42
index=43
index=44
index=45
index=46
index=47
index=48
index=49
index=50
index=51
index=52
index=53
index=54
index=55
index=56
index=57
index=58
index=59
index=60
index=61
index=62
index=63
index=64
index=65
index=66
index=67
index=68
index=69
index=70
index=71
index=72
index=73
index=74
index=75
index=76
index=77
index=78
index=79
index=80
index=81
index=82
index=83
index=84
index=85
index=86
index=87
index=88
index=89
index=90
index=91
index=92
index=93
index=94
index=95
index=96
index=97
index=98
index=99
index=100
index=101
index=102
index=103
index=104
index=105
index=106
index=107
index=108
index=109
index=110


In [6]:
projection_info_df

,Package Name,Component,Unit,Qnty,Cost,Present_Progress,Projected_progress,Package,Haor Type,Structure Code,Earth Volume,Hoar Code,District Code
0,BWDB/Kish/HFMLIP/ PW-01,Betal Doba from KM 1.315 to KM 3.466 =2.151 KM,Km,2.151,40.94,1.00,1.00,KISH/PW-01,Rehab,8,0,R-2,KISH
1,BWDB/Kish/HFMLIP/ PW-01,Alalia-bahadia Khal from KM 1.325 to KM 2.545=...,Km,1.220,35.88,1.00,1.00,KISH/PW-01,Rehab,8,1,R-1,KISH
2,BWDB/Kish/HFMLIP/ PW-01,katakhali khal from KM 0.000 to KM 1.550=1.550 KM,Km,1.550,19.18,0.00,1.00,KISH/PW-01,Rehab,8,2,R-3,KISH
3,BWDB/Kish/HFMLIP/ PW-01,Ganakkhali Khal from KM 0.000 to KM 5.625=5.62...,Km,5.625,309.80,0.00,1.00,KISH/PW-01,Rehab,8,3,R-3,KISH
4,BWDB/Kish/HFMLIP/ PW-01,kali Khal from KM 0.00 to KM 1.000=1.000 KM,Km,1.000,45.95,0.00,1.00,KISH/PW-01,Rehab,8,4,R-3,KISH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,BWDB/Kish/HFMLIP/ PW-27,Kalna,Nos,1.000,148.00,0.00,1.00,KISH/PW-27,New,3,145,N-9,KISH
146,BWDB/Kish/HFMLIP/ PW-27,RCC Box Drainage,Nos,1.000,41.88,0.00,1.00,KISH/PW-27,New,4,146,N-9,KISH
147,BWDB/Kish/HFMLIP/ PW-27,Causeway,Nos,1.000,140.87,0.00,1.00,KISH/PW-27,New,5,147,N-9,KISH
148,BWDB/Kish/HFMLIP/ PW-27,Causeway & Drainage,Nos,1.000,407.31,0.00,1.00,KISH/PW-27,New,5,148,N-9,KISH


In [7]:
dpp_item_df

,DPP_Item,VWD_30_06_2020,VWD_30_06_2021,Payment_30_06_2020,Payment_30_06_2021
0,Inlet,629.833788,646.433215,291.81,354.623215
1,Reg_Rehab_Rehab,0.000000,44.790000,0.00,44.790000
2,Reg_CW_Box,6822.445719,9014.368056,6503.97,2510.398056
3,Khal_Riv_New,3077.582002,4299.901958,2546.54,1753.361958
4,Khal_Riv_Rehab,76.820000,451.750000,51.09,400.660000
5,Full_Emb_Rehab,0.000000,0.000000,0.00,0.000000
6,Sub_Emb_Rehab,0.000000,0.000000,0.00,0.000000
7,Sub_Emb_Const,5929.366851,7264.274679,5833.04,1431.234679
8,Reg_Rehab_New,0.000000,120.556364,0.00,120.556364
9,Thr_Fl_Cost,0.000000,0.000000,0.00,0.000000


In [8]:
print(projection_info_df)

                 Package Name  \
0    BWDB/Kish/HFMLIP/ PW-01    
1    BWDB/Kish/HFMLIP/ PW-01    
2    BWDB/Kish/HFMLIP/ PW-01    
3    BWDB/Kish/HFMLIP/ PW-01    
4    BWDB/Kish/HFMLIP/ PW-01    
..                        ...   
145  BWDB/Kish/HFMLIP/ PW-27    
146  BWDB/Kish/HFMLIP/ PW-27    
147  BWDB/Kish/HFMLIP/ PW-27    
148  BWDB/Kish/HFMLIP/ PW-27    
149   BWDB/Kish/HFMLIP/ PW-28   

                                             Component Unit    Qnty    Cost  \
0       Betal Doba from KM 1.315 to KM 3.466 =2.151 KM   Km   2.151   40.94   
1    Alalia-bahadia Khal from KM 1.325 to KM 2.545=...   Km   1.220   35.88   
2    katakhali khal from KM 0.000 to KM 1.550=1.550 KM   Km   1.550   19.18   
3    Ganakkhali Khal from KM 0.000 to KM 5.625=5.62...   Km   5.625  309.80   
4          kali Khal from KM 0.00 to KM 1.000=1.000 KM   Km   1.000   45.95   
..                                                 ...  ...     ...     ...   
145                                             Ka

In [9]:
package_name_df
structure_type_df
pname=list(package_name_df['Package_No'])
#pname.insert(0,"Unit")
zerolist=[0 for p in pname]
zerolist
stname=list(structure_type_df['Items'])
stname
data={'Package_No':pname}
units=["No","No","No","No","No","No","Km","Km","Km","Km","Km","No","No","No","No"]
for st,unit in zip(stname,units):
    #print(unit)
    #zerolist.pop(0)
    #zerolist.insert(0,unit)   
    data.update({st:zerolist})
print(data)
physical_progress_lfy=pd.DataFrame(data)
physical_progress_cfy=physical_progress_lfy.copy(deep=True)
physical_progress_lfy=buildPhysicalProgressMatrix(projection_info_df,structure_type_df,
                                                  package_name_df,'Present_Progress',physical_progress_lfy)

physical_progress_cfy=buildPhysicalProgressMatrix(projection_info_df,structure_type_df,
                                                  package_name_df,'Projected_progress',physical_progress_cfy)

physical_target_df=buildPhysicalWorkPrgramCurrentYear(physical_progress_cfy,physical_progress_lfy)


myframes.append(physical_progress_lfy)
mynames.append("Physical_30_june_2020")

myframes.append(physical_progress_cfy)
mynames.append("Physical_30_june_2021")
myframes.append(physical_target_df)
mynames.append("Target_30_june_2021")
saveDataFrame(myframes,out_file_name,mynames)


{'Package_No': ['KISH/PW-01', 'KISH/PW-02', 'KISH/PW-03', 'KISH/PW-04', 'KISH/PW-05', 'KISH/PW-06', 'KISH/PW-07', 'KISH/PW-08', 'KISH/PW-09', 'KISH/PW-10', 'KISH/PW-11', 'KISH/PW-12', 'KISH/PW-13', 'KISH/PW-14', 'KISH/PW-15', 'KISH/PW-16', 'KISH/PW-17', 'KISH/PW-18', 'KISH/PW-19', 'KISH/PW-20', 'KISH/PW-21', 'KISH/PW-22', 'KISH/PW-23', 'KISH/PW-24', 'KISH/PW-25', 'KISH/PW-26', 'KISH/PW-27', 'KISH/PW-28/Lot-01', 'KISH/PW-28/Lot-02', 'KISH/PW-28/Lot-03', 'KISH/PW-28/Lot-04', 'KISH/PW-29', 'KISH/PW-30', 'KISH/PW-31', 'KISH/PW-32', 'KISH/PW-33', 'KISH/PW-34', 'KISH/PW-35', 'KISH/PW-36', 'KISH/PW-37', 'KISH/PW-38', 'HOBI/PW-01', 'HOBI/PW-02', 'HOBI/PW-03', 'HOBI/PW-04', 'HOBI/PW-05', 'HOBI/PW-06', 'HOBI/PW-07', 'NETR/PW-01', 'NETR/PW-02', 'NETR/PW-03', 'NETR/PW-04', 'NETR/PW-05', 'NETR/PW-06', 'NETR/PW-07', 'NETR/PW-08', 'SUNM/PW-01', 'SUNM/PW-02', 'SUNM/PW-03', 'SUNM/PW-04', 'SUNM/PW-05', 'SUNM/PW-06', 'SUNM/PW-07'], 'Construction of Irrigation Inlet': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 